In [10]:
import numpy as np
import random

In [11]:
# meal dataset
meal_data = [
    {"name": "Rice and Curry", "calories": 450, "protein": 20, "carbs": 55, "fats": 10},
    {"name": "Hoppers", "calories": 400, "protein": 10, "carbs": 50, "fats": 8},
    {"name": "Fish Curry", "calories": 350, "protein": 30, "carbs": 5, "fats": 12},
    {"name": "Cheese Pasta", "calories": 500, "protein": 25, "carbs": 60, "fats": 20},
    {"name": "Grilled Chicken", "calories": 300, "protein": 40, "carbs": 10, "fats": 8},
    {"name": "Vegetable Salad", "calories": 250, "protein": 15, "carbs": 40, "fats": 5},
    {"name": "Burger and Fries", "calories": 800, "protein": 30, "carbs": 90, "fats": 25},
    {"name": "String Hoppers", "calories": 600, "protein": 15, "carbs": 70, "fats": 20},
]

In [12]:
# user preferences and goals
goals = {
    "calories": 500,
    "protein": 30,
    "carbs": 50,
    "fats": 10,
}

In [13]:
# Objective function
def objective_function(meal_indices):
    total = {
        "calories": 0,
        "protein": 0,
        "carbs": 0,
        "fats": 0,
    }

    for idx in meal_indices:
        meal = meal_data[idx]
        for key in total:
            total[key] += meal[key]

    # Calculate the weighted difference from the goal
    score = sum(abs(total[key] - goals[key]) for key in goals)
    return score

In [14]:
# Particle class
class Particle:
    def __init__(self, num_meals, n_particles):
        self.position = random.sample(range(num_meals), n_particles)  # Unique meal indices
        self.velocity = [0] * n_particles
        self.best_position = list(self.position)
        self.best_score = float('inf')

In [15]:
# PSO Algorithm
def particle_swarm_optimization(objective, num_meals, n_particles, num_particles, num_iterations):
    particles = [Particle(num_meals, n_particles) for _ in range(num_particles)]
    global_best_position = None
    global_best_score = float('inf')

    w = 0.5  # inertia weight
    c1 = 1   # cognitive parameter
    c2 = 2   # social parameter

    for _ in range(num_iterations):
        for particle in particles:
            score = objective(particle.position)
            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = list(particle.position)

            if score < global_best_score:
                global_best_score = score
                global_best_position = list(particle.position)

        for particle in particles:
            r1, r2 = np.random.rand(2)
            for i in range(len(particle.position)):
                cognitive_component = c1 * r1 * (particle.best_position[i] - particle.position[i])
                social_component = c2 * r2 * (global_best_position[i] - particle.position[i])
                particle.velocity[i] = w * particle.velocity[i] + cognitive_component + social_component
                particle.position[i] = int((particle.position[i] + particle.velocity[i]) % num_meals)

            # unique meal indices
            particle.position = list(set(particle.position))
            while len(particle.position) < n_particles:
                candidate = random.choice(range(num_meals))
                if candidate not in particle.position:
                    particle.position.append(candidate)

    return global_best_position, global_best_score

In [16]:
num_meals = len(meal_data)
n_particles = 3  # Number of meals in a plan
num_particles = 30
num_iterations = 100

In [17]:
# Run PSO
best_positions, best_score = particle_swarm_optimization(objective_function, num_meals, n_particles, num_particles, num_iterations)

print("Optimal Meal Plan:")
for idx in best_positions:
    meal = meal_data[idx]
    print(f"{meal['name']} - Calories: {meal['calories']}, Protein: {meal['protein']}g, Carbs: {meal['carbs']}g, Fats: {meal['fats']}g")

print(f"Optimal Score: {best_score:.2f}")

Optimal Meal Plan:
Fish Curry - Calories: 350, Protein: 30g, Carbs: 5g, Fats: 12g
Grilled Chicken - Calories: 300, Protein: 40g, Carbs: 10g, Fats: 8g
Vegetable Salad - Calories: 250, Protein: 15g, Carbs: 40g, Fats: 5g
Optimal Score: 475.00
